In [1]:
#from astropy.io import fits
import numpy as np
#import glob
#import os
import pickle
import matplotlib.pyplot as plt
import astropy
import pandas as pd
#from astropy.io import fits
from scipy.optimize import curve_fit

from photutils.centroids import (centroid_1dg, centroid_2dg,
                                 centroid_com, centroid_quadratic)

%matplotlib qt

In [2]:
# retrieve processed data arrays from pickle files

# single fiber
file = open('yoo_jung_single.pkl', 'rb')
dict_single = pickle.load(file)
file.close()

# 3 photonic lantern
file = open('yoo_jung_3PL.pkl', 'rb')
dict_3PL = pickle.load(file)
file.close()

# broadband image
file = open('yoo_jung_broadband.pkl', 'rb')
dict_broadband = pickle.load(file)
file.close()

In [3]:
# retrieve centroids as fcn of wavelength

file = open('single_wavel_centroids.pkl', 'rb')
dict_centroids = pickle.load(file)
file.close()

In [4]:
# convert to DataFrame 

df_centroids = pd.DataFrame(columns=['wavel','x','y'])

for key, val in dict_centroids.items():
    
    dict_this = {'wavel':float(key),'x':val[0],'y':val[1]}
    
    df_centroids = df_centroids.append(dict_this, ignore_index=True)

/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_20621/2637175356.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_centroids = df_centroids.append(dict_this, ignore_index=True)
/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_20621/2637175356.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_centroids = df_centroids.append(dict_this, ignore_index=True)
/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_20621/2637175356.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_centroids = df_centroids.append(dict_this, ignore_index=True)
/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_20621/2637175356.py:9: FutureWarning: The frame.append method is deprecated and will be removed

In [21]:
## ## TEMPORARY KLUDGE TO REMOVE A BAD FIT
df_centroids = df_centroids.drop(index=19) #sort_values(by='wavel')

In [22]:
def func(X, a, b, c, d, f):
    
    x_pass, y_pass = X
    
    return a*x_pass + b*y_pass + c*x_pass*y_pass + d*np.power(x_pass,2.) + f*np.power(y_pass,2.)

x = df_centroids['x'].values
y = df_centroids['y'].values
z = df_centroids['wavel'].values

# initial guesses for a,b,c:
p0 = 1., 1., 1., 1., 1.
fit_coeffs = curve_fit(func, (x,y), z, p0)

In [23]:
fit_coeffs

(array([-2.14798392e+01,  4.89453386e+01,  1.63578627e-01,  6.38728496e-03,
        -3.11863017e-01]),
 array([[ 5.75470912e+02, -9.61415875e+02, -4.43713025e+00,
          3.11516021e-02,  7.32583082e+00],
        [-9.61415875e+02,  1.85825249e+03,  7.42749995e+00,
         -5.24460844e-02, -1.41598528e+01],
        [-4.43713025e+00,  7.42749995e+00,  3.42139216e-02,
         -2.40528681e-04, -5.65968311e-02],
        [ 3.11516021e-02, -5.24460844e-02, -2.40528681e-04,
          1.80384708e-06,  3.99787576e-04],
        [ 7.32583082e+00, -1.41598528e+01, -5.65968311e-02,
          3.99787576e-04,  1.07898174e-01]]))

In [24]:
# column of best-fit wavelength values

df_centroids['wavel_bestfit'] = func((x,y), fit_coeffs[0][0], fit_coeffs[0][1], fit_coeffs[0][2], 
                                     fit_coeffs[0][3], fit_coeffs[0][4])

In [15]:
#plt.scatter(df_centroids['x'],df_centroids['y'])
plt.scatter(df_centroids['x'],df_centroids['y'],c=df_centroids['wavel_bestfit'])
#plt.scatter(df_centroids['y'],df_centroids['wavel_bestfit'])
plt.colorbar()
plt.show()

In [8]:
plt.scatter(df_centroids['wavel'],df_centroids['wavel']-df_centroids['wavel_bestfit'])
plt.ylabel('True minus best fit lambda')
plt.xlabel('True lambda')
plt.show()

In [9]:
plt.scatter(df_centroids['wavel'],df_centroids['wavel_bestfit'])
plt.plot([1000,1700], [1000,1700], linestyle='--')
plt.ylabel('Best fit lambda')
plt.xlabel('True lambda')
plt.show()

In [25]:
# pickle

file = open('junk_wavel_soln.pkl', 'wb')

# dump information to that file
pickle.dump(fit_coeffs, file)
# close the file
file.close()